<a href="https://colab.research.google.com/github/Venkat3103/fantasy-football/blob/main/Compute_Teams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##INSTALL / IMPORT REQUIRED LIBRARIES

In [1]:
!pip install pulp

     |████████████████████████████████| 40.6MB 74kB/s 


In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog
from scipy.stats import linregress
from pulp import *

In [ ]:
#from google.colab import files
#uploaded = files.upload()

Saving EPL_2020_21.csv to EPL_2020_21.csv


In [ ]:
#import io
#epl=pd.read_csv(io.BytesIO(uploaded['EPL_2020_21.csv']))

# IMPORT DATA SETS FROM LOCAL COMPUTER

## Import League Dataset

In [328]:
from google.colab import files
uploaded = files.upload()

Saving SPNvsSUI.xlsx to SPNvsSUI (1).xlsx


In [329]:
import io
league_data=pd.read_excel(io.BytesIO(uploaded['SPNvsSUI.xlsx']))
league_data

,name,pos_details,total_shots,shots_on_target,total_passes,key_passes,appearances,assists,tackles_won,interceptions,saves,red_cards,yellow_cards,own_goals,total_goals,tournament,mins_played,country
0,Harry Kane,striker,137,53,908,49,35,14,22,11,0,0,1,0,23,Premier League,3087,ENG
1,Kevin De Bruyne,midfielder,80,26,1265,80,25,12,34,9,0,0,1,0,6,Premier League,2001,NaN
2,Jack Grealish,midfielder,50,19,996,81,26,10,31,12,0,0,6,0,6,Premier League,2184,ENG
3,Bruno Fernandes,midfielder,121,51,2112,95,37,12,57,26,0,0,6,0,18,Premier League,3110,NaN
4,Tomas Soucek,midfielder,64,23,1607,18,38,1,78,62,0,1,7,1,10,Premier League,3420,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1623,Remo Freuler,midfielder,20,7,2079,30,32,3,63,39,0,0,5,0,2,SerieA,2721,SUI
1624,Álvaro Morata,striker,56,24,575,40,23,9,17,5,0,1,3,0,11,SerieA,2019,SPN
1625,Haris Seferovic,striker,92,35,521,23,24,7,4,2,0,0,2,0,22,LigaNOS,2266,SUI
1626,Ricardo Rodríguez,defender,13,2,636,6,15,0,21,20,0,0,1,0,0,SerieA,1245,SUI


## Import Dream11 Dataset

In [434]:
from google.colab import files
uploaded = files.upload()

Saving SPNvsSUI_DREAM11.xlsx to SPNvsSUI_DREAM11 (2).xlsx


In [454]:
dream11_data=pd.read_excel(io.BytesIO(uploaded['SPNvsSUI_DREAM11.xlsx']))
dream_name=dream11_data.name.tolist()
dream_name


['Unai Simón',
 'Yann Sommer',
 'David de Gea',
 'Yvon Mvogo',
 'Gregor Kobel',
 'Jonas Omlin',
 'Robert Sánchez',
 'César Azpilicueta',
 'Fabian Schär',
 'Aymeric Laporte',
 'Jordi Alba',
 'Pau Torres',
 'Manuel Akanji',
 'Kevin Mbabu',
 'Nico Elvedi',
 'Eric García',
 'Marcos Llorente',
 'Ricardo Rodríguez',
 'José Gayà',
 'Silvan Widmer',
 'Loris Benito',
 'Bećir Omeragić',
 'Diego Llorente',
 'Eray Cömert',
 'Jordan Lotomba',
 'Xherdan Shaqiri',
 'Pablo Sarabia',
 'Dani Olmo',
 'Steven Zuber',
 'Adama Traoré',
 'Ruben Vargas',
 'Sergio Busquets',
 'Koke',
 'Granit Xhaka',
 'Thiago Alcantara',
 'Denis Zakaria',
 'Edimilson Fernandes',
 'Rodrigo Hernandez',
 'Remo Freuler',
 'Fabián Ruiz Pena',
 'Christian Fassnacht',
 'Pedro Gonzalez',
 'Djibril Sow',
 'Haris Seferovic',
 'Álvaro Morata',
 'Ferran Torres',
 'Mario Gavranović',
 'Admir Mehmedi',
 'Breel Embolo',
 'Gerard Moreno',
 'Mikel Oyarzabal']

In [455]:
dream11_data['avg_time_dream11'] = dream11_data['mins_played']/dream11_data['mat_played']
dream11_data['avg_time_dream11'] = dream11_data.avg_time_dream11.fillna(0);
#dream11_data = dream11_data[dream11_data['avg_time_dream11']>30]
dream11_data

,name,epl_points,epl_value,pos_detail,club,mat_played,is_playing,mins_played,avg_time_dream11
0,Unai Simón,99,8.5,goalkeeper,SPN,4,1,390,97.500000
1,Yann Sommer,82,8.5,goalkeeper,SUI,4,1,390,97.500000
2,David de Gea,0,8.5,goalkeeper,SPN,0,0,0,0.000000
3,Yvon Mvogo,0,8.5,goalkeeper,SUI,0,0,0,0.000000
4,Gregor Kobel,0,8.0,goalkeeper,SUI,0,0,0,0.000000
5,Jonas Omlin,0,8.0,goalkeeper,SUI,0,0,0,0.000000
6,Robert Sánchez,0,8.0,goalkeeper,SPN,0,0,0,0.000000
7,César Azpilicueta,130,9.0,defender,SPN,2,1,197,98.500000
8,Fabian Schär,35,9.0,defender,SUI,3,0,170,56.666667
9,Aymeric Laporte,205,8.5,defender,SPN,4,1,390,97.500000


## Assign Clubs for the day's game

In [456]:
clubs = dream11_data.club.unique()
club1 = clubs[0]
club2 = clubs[1]

##Obtain Particular Match From League Dataset

In [457]:
league_data=league_data[(league_data['country']==club1)|(league_data['country']==club2)]
league_data.reset_index(drop=True,inplace=True)
league_data

,name,pos_details,total_shots,shots_on_target,total_passes,key_passes,appearances,assists,tackles_won,interceptions,saves,red_cards,yellow_cards,own_goals,total_goals,tournament,mins_played,country,dream_points
0,Adama Traoré,striker,43,13,700,55,37,2,28,7,0,0,4,0,2,Premier League,2645,SPN,1073.0
1,César Azpilicueta,defender,11,5,1887,27,26,2,53,28,0,1,5,0,1,Premier League,2191,SPN,1028.4
2,Aymeric Laporte,defender,6,1,1274,4,16,0,13,11,0,0,1,0,0,Premier League,1343,SPN,404.8
3,Ferran Torres,striker,36,15,406,19,24,2,12,3,0,0,1,0,7,Premier League,1306,SPN,930.2
4,Granit Xhaka,midfielder,21,7,2188,17,31,2,50,24,0,1,7,0,1,Premier League,2522,SUI,1114.6
5,David de Gea,goalkeeper,0,0,501,0,26,0,0,0,60,0,0,0,0,Premier League,2296,SPN,100.2
6,Xherdan Shaqiri,midfielder,11,0,399,19,14,2,7,2,0,0,2,0,0,Premier League,553,SUI,210.8
7,Fabian Schär,defender,12,5,581,0,18,0,13,27,0,1,4,0,1,Premier League,1247,SUI,480.2
8,Nico Elvedi,defender,12,6,1895,3,29,0,49,65,0,0,1,0,3,Bundesliga,2538,SUI,1116.0
9,Kevin Mbabu,defender,11,5,791,14,22,1,51,22,0,0,7,0,0,Bundesliga,1646,SUI,717.2


##Calculate Dream 11 Points From League Data

ATTACK:
Goal
Scored by a striker
+40
Scored by a mid-fielder
+50
Scored by a defender or goalkeeper
+60
Assist
+20
Shot on Target
Includes Goals
+6
Chance Created
The final pass leading to a shot (on target including goals, blocked or off target)
+3
5 Passes Completed
+1
DEFENSE:
Tackle Won
+4
Interception Won
+4
Saves
GK
+6
Penalty Saved
GK
+50
Clean Sheet **
GK/DEF (Played +55 mins)
+20
OTHERS:
Captain
2x
Vice-Captain
1.5x
In Starting 11
+4
Coming on as a Substitute
+2
PENALTIES:
Yellow card
-4
Red card
-10
Own goal
-8
Goals conceded
GK/DEF (on the field when the goal is scored)
-2
Penalty missed 
-20


In [458]:
#true data is match data created using the league data
#final_league_data is the match data with the past league data computed over the true_data
league_data['dream_points']=0

league_data['dream_points']=league_data['dream_points']+league_data['shots_on_target']*40+league_data['assists']*20+league_data['shots_on_target']*6+(league_data['key_passes']+league_data['assists'])*3+(league_data['total_passes']/5)*1+league_data['tackles_won']*4+league_data['interceptions']*4+league_data['yellow_cards']*(-4)+league_data['red_cards']*(-10)+league_data['own_goals']*(-8)

final_league_data = pd.DataFrame()
df_s=league_data[league_data['pos_details']=="striker"]

df_m=league_data[league_data['pos_details']=="midfielder"]
df_d=league_data[(league_data['pos_details']=="defender")|(league_data['pos_details']=="goalkeeper")]
df_s['dream_points']=(df_s['dream_points']+df_s['total_goals']*40)
df_m['dream_points']=(df_m['dream_points']+df_m['total_goals']*50)
df_d['dream_points']=(df_d['dream_points']+df_d['total_goals']*60)

final_league_data=pd.concat([final_league_data,df_s])
final_league_data=pd.concat([final_league_data,df_m])
final_league_data=pd.concat([final_league_data,df_d])
final_league_data.reset_index(inplace=True,drop=True)

final_league_data


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,name,pos_details,total_shots,shots_on_target,total_passes,key_passes,appearances,assists,tackles_won,interceptions,saves,red_cards,yellow_cards,own_goals,total_goals,tournament,mins_played,country,dream_points
0,Adama Traoré,striker,43,13,700,55,37,2,28,7,0,0,4,0,2,Premier League,2645,SPN,1153.0
1,Ferran Torres,striker,36,15,406,19,24,2,12,3,0,0,1,0,7,Premier League,1306,SPN,1210.2
2,Breel Embolo,striker,41,21,439,16,31,2,15,8,0,0,4,0,5,Bundesliga,1435,SUI,1423.8
3,Gerard Moreno,striker,97,47,1056,44,33,7,24,15,0,0,3,0,23,LaLiga,2683,SPN,3730.2
4,Mikel Oyarzabal,striker,62,27,812,51,33,8,30,15,0,0,1,0,11,LaLiga,2432,SPN,2357.4
5,Álvaro Morata,striker,56,24,575,40,23,9,17,5,0,1,3,0,11,SerieA,2019,SPN,2052.0
6,Haris Seferovic,striker,92,35,521,23,24,7,4,2,0,0,2,0,22,LigaNOS,2266,SUI,2840.2
7,Granit Xhaka,midfielder,21,7,2188,17,31,2,50,24,0,1,7,0,1,Premier League,2522,SUI,1164.6
8,Xherdan Shaqiri,midfielder,11,0,399,19,14,2,7,2,0,0,2,0,0,Premier League,553,SUI,210.8
9,Dani Olmo,midfielder,77,24,1217,41,32,9,23,13,0,0,1,0,5,Bundesliga,2207,SPN,2067.4


## Set Constraint that Average time played per match so far must be >30


In [459]:
dream11_data
dream11_data = dream11_data[dream11_data['avg_time_dream11']>30]
dream11_data.reset_index(drop=True,inplace=True)
dream11_data

,name,epl_points,epl_value,pos_detail,club,mat_played,is_playing,mins_played,avg_time_dream11
0,Unai Simón,99,8.5,goalkeeper,SPN,4,1,390,97.500000
1,Yann Sommer,82,8.5,goalkeeper,SUI,4,1,390,97.500000
2,César Azpilicueta,130,9.0,defender,SPN,2,1,197,98.500000
3,Fabian Schär,35,9.0,defender,SUI,3,0,170,56.666667
4,Aymeric Laporte,205,8.5,defender,SPN,4,1,390,97.500000
5,Jordi Alba,164,8.5,defender,SPN,4,1,312,78.000000
6,Pau Torres,134,8.5,defender,SPN,4,1,247,61.750000
7,Manuel Akanji,90,8.5,defender,SPN,4,1,390,97.500000
8,Kevin Mbabu,75,8.5,defender,SUI,4,0,196,49.000000
9,Nico Elvedi,73,8.5,defender,SUI,4,1,390,97.500000


# Merge Dataframes

In [460]:
temp_data = pd.merge(dream11_data,final_league_data,on='name',how='left');
#temp_data
final_data = temp_data[['name','epl_value','pos_detail','club','mins_played_x','epl_points','mins_played_y','dream_points',"is_playing"]]
final_data.rename(columns = {'mins_played_x' : 'mins_played_dream11' , 'mins_played_y' : 'mins_played_league', 'epl_points' : 'dream11_points', 'epl_value' : 'credits' }, inplace=True)
final_data.sort_values(by="pos_detail",ascending=True)
final_data['mins_played_league'] = final_data.mins_played_league.fillna(0);
final_data['dream_points'] = final_data.dream_points.fillna(0);
final_data

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

,name,credits,pos_detail,club,mins_played_dream11,dream11_points,mins_played_league,dream_points,is_playing
0,Unai Simón,8.5,goalkeeper,SPN,390,99,3330.0,174.8,1
1,Yann Sommer,8.5,goalkeeper,SUI,390,82,2713.0,221.0,1
2,César Azpilicueta,9.0,defender,SPN,197,130,2191.0,1088.4,1
3,Fabian Schär,9.0,defender,SUI,170,35,1247.0,540.2,0
4,Aymeric Laporte,8.5,defender,SPN,390,205,1343.0,404.8,1
5,Jordi Alba,8.5,defender,SPN,312,164,3030.0,1759.8,1
6,Pau Torres,8.5,defender,SPN,247,134,2970.0,877.6,1
7,Manuel Akanji,8.5,defender,SPN,390,90,2350.0,1126.6,1
8,Kevin Mbabu,8.5,defender,SUI,196,75,1646.0,717.2,0
9,Nico Elvedi,8.5,defender,SUI,390,73,2538.0,1296.0,1


#INCLUDING ONLY PLAYERS IN THE LINE UP

In [461]:
final_data = final_data[final_data['is_playing']==1]

In [462]:
final_data.reset_index(inplace=True,drop=True)
final_data

,name,credits,pos_detail,club,mins_played_dream11,dream11_points,mins_played_league,dream_points,is_playing
0,Unai Simón,8.5,goalkeeper,SPN,390,99,3330.0,174.8,1
1,Yann Sommer,8.5,goalkeeper,SUI,390,82,2713.0,221.0,1
2,César Azpilicueta,9.0,defender,SPN,197,130,2191.0,1088.4,1
3,Aymeric Laporte,8.5,defender,SPN,390,205,1343.0,404.8,1
4,Jordi Alba,8.5,defender,SPN,312,164,3030.0,1759.8,1
5,Pau Torres,8.5,defender,SPN,247,134,2970.0,877.6,1
6,Manuel Akanji,8.5,defender,SPN,390,90,2350.0,1126.6,1
7,Nico Elvedi,8.5,defender,SUI,390,73,2538.0,1296.0,1
8,Ricardo Rodríguez,8.5,defender,SUI,357,46,1245.0,397.2,1
9,Silvan Widmer,8.5,defender,SUI,194,40,0.0,0.0,1


# ADDING WEIGHTS


100 DREAM, 0 LEAGUE

In [463]:
D100L0 = final_data.copy()
D100L0['ppm'] = ((1*final_data['dream11_points']) + (0*final_data['dream_points'])) / ((1*(final_data['mins_played_dream11'])/90) + (0*((final_data['mins_played_league'])/90)))
D100L0['ppm'] = D100L0['ppm'].fillna(0)
D100L0

,name,credits,pos_detail,club,mins_played_dream11,dream11_points,mins_played_league,dream_points,is_playing,ppm
0,Unai Simón,8.5,goalkeeper,SPN,390,99,3330.0,174.8,1,22.846154
1,Yann Sommer,8.5,goalkeeper,SUI,390,82,2713.0,221.0,1,18.923077
2,César Azpilicueta,9.0,defender,SPN,197,130,2191.0,1088.4,1,59.390863
3,Aymeric Laporte,8.5,defender,SPN,390,205,1343.0,404.8,1,47.307692
4,Jordi Alba,8.5,defender,SPN,312,164,3030.0,1759.8,1,47.307692
5,Pau Torres,8.5,defender,SPN,247,134,2970.0,877.6,1,48.825911
6,Manuel Akanji,8.5,defender,SPN,390,90,2350.0,1126.6,1,20.769231
7,Nico Elvedi,8.5,defender,SUI,390,73,2538.0,1296.0,1,16.846154
8,Ricardo Rodríguez,8.5,defender,SUI,357,46,1245.0,397.2,1,11.596639
9,Silvan Widmer,8.5,defender,SUI,194,40,0.0,0.0,1,18.556701


In [464]:
#lp for finding team
data = D100L0.copy();
player = {str(i): data['name'][i] for i in range(data.shape[0])}
point = {str(i): data['ppm'][i] for i in range(data.shape[0])} 
cost = {str(i): data['credits'][i] for i in range(data.shape[0])}
gk = {str(i): 1 if data['pos_detail'][i] == 'goalkeeper' else 0 for i in range(data.shape[0])}
defender = {str(i): 1 if data['pos_detail'][i] == 'defender' else 0 for i in range(data.shape[0])}
midfielder = {str(i): 1 if data['pos_detail'][i] == 'midfielder' else 0 for i in range(data.shape[0])}
striker = {str(i): 1 if data['pos_detail'][i] == 'striker' else 0 for i in range(data.shape[0])}
club1 = {str(i): 1 if data['club'][i] == club1 else 0 for i in range(data.shape[0])}
club2 = {str(i): 1 if data['club'][i] == club2 else 0 for i in range(data.shape[0])}
xi = {str(i): 1 for i in range(data.shape[0])}

prob = LpProblem("Fantasy Football",LpMaximize)

player_index = LpVariable.dicts("Players",player,0,1,LpBinary)

# objective function
prob += lpSum([point[i]*player_index[i] for i in player]), "Total Cost"

# constraints
prob += lpSum([player_index[i] for i in player]) == 11, "Total 11 Players"
prob += lpSum([cost[i] * player_index[i] for i in player]) <= 100.0, "Total Cost"
prob += lpSum([gk[i] * player_index[i] for i in player]) == 1, "Only 1 GK"
prob += lpSum([defender[i] * player_index[i] for i in player]) >= 3, "Min 3 DEF"
prob += lpSum([defender[i] * player_index[i] for i in player]) <= 5, "Max 5 DEF"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) >= 3, "Min 3 MID"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) <= 5, "Max 5 MID"
prob += lpSum([striker[i] * player_index[i] for i in player]) >= 1, "Min 1 STR"
prob += lpSum([striker[i] * player_index[i] for i in player]) <= 3, "Max 3 STR"
prob += lpSum([club1[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club1"
prob += lpSum([club2[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club2"

# solve
status = prob.solve()
print(status)

1


/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [465]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [466]:
selection = {}
for v in prob.variables():
    index = int(v.name.split("_")[1])
    selection[index] = v.varValue

In [467]:
data['integer_programming'] = 0.0
for i in selection:
    data.loc[i, 'integer_programming'] = selection[i]

In [468]:
XI = data[data['integer_programming'] == 1.0]
TOP2 = XI['ppm'].nlargest(2)
CAPTAIN_POINTS = TOP2.iloc[0]
CAPTAIN = XI['name'][XI.ppm==CAPTAIN_POINTS]
VICECAPTAIN_POINTS = TOP2.iloc[1]
VICECAPTAIN = XI['name'][XI.ppm==VICECAPTAIN_POINTS]
PLAYER_POINTS = XI['ppm'].sum() 
TOTAL_POINTS = PLAYER_POINTS + CAPTAIN_POINTS + VICECAPTAIN_POINTS/2
TOTAL_COST = XI['credits'].sum()
TOTAL_PLAYERS = XI.shape[0]
print("Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

Total Points: 753.1358969681256, Raw Player Points: 617.2121492042616, Cost: 99.0, Captain: Pablo Sarabia, Vice-Captain: Ferran Torres


In [469]:
print("100 DREAM : 0 LEAGUE")
print(XI[['name','pos_detail','club','credits','ppm',]]
 .sort_values(['pos_detail'])
 .reset_index(drop=True))

print("\n Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

100 DREAM : 0 LEAGUE
                 name  pos_detail club  credits        ppm
0   César Azpilicueta    defender  SPN      9.0  59.390863
1     Aymeric Laporte    defender  SPN      8.5  47.307692
2          Jordi Alba    defender  SPN      8.5  47.307692
3          Pau Torres    defender  SPN      8.5  48.825911
4         Yann Sommer  goalkeeper  SUI      8.5  18.923077
5     Xherdan Shaqiri  midfielder  SUI      9.5  62.474227
6       Pablo Sarabia  midfielder  SPN      9.5  99.086538
7        Steven Zuber  midfielder  SUI      9.0  65.786802
8     Sergio Busquets  midfielder  SPN      8.5  34.335260
9     Haris Seferovic     striker  SUI     10.0  60.099668
10      Ferran Torres     striker  SPN      9.5  73.674419

 Total Points: 753.1358969681256, Raw Player Points: 617.2121492042616, Cost: 99.0, Captain: Pablo Sarabia, Vice-Captain: Ferran Torres


0 DREAM 100 LEAGUE


In [470]:
D0L100 = final_data.copy()
D0L100['ppm'] = ((0*final_data['dream11_points']) + (1*final_data['dream_points'])) / ((0*(final_data['mins_played_dream11'])/90) + (1*((final_data['mins_played_league'])/90)))
D0L100['ppm'] = D0L100['ppm'].fillna(0)
D0L100

,name,credits,pos_detail,club,mins_played_dream11,dream11_points,mins_played_league,dream_points,is_playing,ppm
0,Unai Simón,8.5,goalkeeper,SPN,390,99,3330.0,174.8,1,4.724324
1,Yann Sommer,8.5,goalkeeper,SUI,390,82,2713.0,221.0,1,7.331367
2,César Azpilicueta,9.0,defender,SPN,197,130,2191.0,1088.4,1,44.708352
3,Aymeric Laporte,8.5,defender,SPN,390,205,1343.0,404.8,1,27.127327
4,Jordi Alba,8.5,defender,SPN,312,164,3030.0,1759.8,1,52.271287
5,Pau Torres,8.5,defender,SPN,247,134,2970.0,877.6,1,26.593939
6,Manuel Akanji,8.5,defender,SPN,390,90,2350.0,1126.6,1,43.146383
7,Nico Elvedi,8.5,defender,SUI,390,73,2538.0,1296.0,1,45.957447
8,Ricardo Rodríguez,8.5,defender,SUI,357,46,1245.0,397.2,1,28.713253
9,Silvan Widmer,8.5,defender,SUI,194,40,0.0,0.0,1,0.000000


In [471]:
#lp for finding team
data = D0L100.copy();
player = {str(i): data['name'][i] for i in range(data.shape[0])}
point = {str(i): data['ppm'][i] for i in range(data.shape[0])} 
cost = {str(i): data['credits'][i] for i in range(data.shape[0])}
gk = {str(i): 1 if data['pos_detail'][i] == 'goalkeeper' else 0 for i in range(data.shape[0])}
defender = {str(i): 1 if data['pos_detail'][i] == 'defender' else 0 for i in range(data.shape[0])}
midfielder = {str(i): 1 if data['pos_detail'][i] == 'midfielder' else 0 for i in range(data.shape[0])}
striker = {str(i): 1 if data['pos_detail'][i] == 'striker' else 0 for i in range(data.shape[0])}
club1 = {str(i): 1 if data['club'][i] == club1 else 0 for i in range(data.shape[0])}
club2 = {str(i): 1 if data['club'][i] == club2 else 0 for i in range(data.shape[0])}
xi = {str(i): 1 for i in range(data.shape[0])}

prob = LpProblem("Fantasy Football",LpMaximize)

player_index = LpVariable.dicts("Players",player,0,1,LpBinary)

# objective function
prob += lpSum([point[i]*player_index[i] for i in player]), "Total Cost"

# constraints
prob += lpSum([player_index[i] for i in player]) == 11, "Total 11 Players"
prob += lpSum([cost[i] * player_index[i] for i in player]) <= 100.0, "Total Cost"
prob += lpSum([gk[i] * player_index[i] for i in player]) == 1, "Only 1 GK"
prob += lpSum([defender[i] * player_index[i] for i in player]) >= 3, "Min 3 DEF"
prob += lpSum([defender[i] * player_index[i] for i in player]) <= 5, "Max 5 DEF"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) >= 3, "Min 3 MID"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) <= 5, "Max 5 MID"
prob += lpSum([striker[i] * player_index[i] for i in player]) >= 1, "Min 1 STR"
prob += lpSum([striker[i] * player_index[i] for i in player]) <= 3, "Max 3 STR"
prob += lpSum([club1[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club1"
prob += lpSum([club2[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club2"

# solve
status = prob.solve()
print(status)

1


/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [472]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [473]:
selection = {}
for v in prob.variables():
    index = int(v.name.split("_")[1])
    selection[index] = v.varValue

In [474]:
data['integer_programming'] = 0.0
for i in selection:
    data.loc[i, 'integer_programming'] = selection[i]

In [475]:
XI = data[data['integer_programming'] == 1.0]
TOP2 = XI['ppm'].nlargest(2)
CAPTAIN_POINTS = TOP2.iloc[0]
CAPTAIN = XI['name'][XI.ppm==CAPTAIN_POINTS]
VICECAPTAIN_POINTS = TOP2.iloc[1]
VICECAPTAIN = XI['name'][XI.ppm==VICECAPTAIN_POINTS]
PLAYER_POINTS = XI['ppm'].sum() 
TOTAL_POINTS = PLAYER_POINTS + CAPTAIN_POINTS + VICECAPTAIN_POINTS/2
TOTAL_COST = XI['credits'].sum()
TOTAL_PLAYERS = XI.shape[0]
print("Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

Total Points: 858.9594846718137, Raw Player Points: 693.8672923308469, Cost: 99.0, Captain: Haris Seferovic, Vice-Captain: Pablo Sarabia


In [476]:
print("0 DREAM : 100 LEAGUE")
print(XI[['name','pos_detail','club','credits','ppm',]]
 .sort_values(['pos_detail'])
 .reset_index(drop=True))

print("\n Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

0 DREAM : 100 LEAGUE
                 name  pos_detail club  credits         ppm
0   César Azpilicueta    defender  SPN      9.0   44.708352
1          Jordi Alba    defender  SPN      8.5   52.271287
2         Nico Elvedi    defender  SUI      8.5   45.957447
3         Yann Sommer  goalkeeper  SUI      8.5    7.331367
4       Pablo Sarabia  midfielder  SPN      9.5  104.572734
5        Steven Zuber  midfielder  SUI      9.0   56.412371
6     Sergio Busquets  midfielder  SPN      8.5   43.235573
7        Remo Freuler  midfielder  SUI      8.5   45.803749
8     Haris Seferovic     striker  SUI     10.0  112.805825
9       Álvaro Morata     striker  SPN     10.0   91.471025
10       Breel Embolo     striker  SUI      9.0   89.297561

 Total Points: 858.9594846718137, Raw Player Points: 693.8672923308469, Cost: 99.0, Captain: Haris Seferovic, Vice-Captain: Pablo Sarabia


50 DREAM 50 LEAGUE

In [477]:
D50L50 = final_data.copy()
D50L50['ppm'] = ((0.50*final_data['dream11_points']) + (0.50*final_data['dream_points'])) / ((0.50*(final_data['mins_played_dream11'])/90) + (0.50*((final_data['mins_played_league'])/90)))
D50L50['ppm'] = D50L50['ppm'].fillna(0)
D50L50

,name,credits,pos_detail,club,mins_played_dream11,dream11_points,mins_played_league,dream_points,is_playing,ppm
0,Unai Simón,8.5,goalkeeper,SPN,390,99,3330.0,174.8,1,6.624194
1,Yann Sommer,8.5,goalkeeper,SUI,390,82,2713.0,221.0,1,8.788269
2,César Azpilicueta,9.0,defender,SPN,197,130,2191.0,1088.4,1,45.919598
3,Aymeric Laporte,8.5,defender,SPN,390,205,1343.0,404.8,1,31.668782
4,Jordi Alba,8.5,defender,SPN,312,164,3030.0,1759.8,1,51.807899
5,Pau Torres,8.5,defender,SPN,247,134,2970.0,877.6,1,28.300901
6,Manuel Akanji,8.5,defender,SPN,390,90,2350.0,1126.6,1,39.961314
7,Nico Elvedi,8.5,defender,SUI,390,73,2538.0,1296.0,1,42.079918
8,Ricardo Rodríguez,8.5,defender,SUI,357,46,1245.0,397.2,1,24.898876
9,Silvan Widmer,8.5,defender,SUI,194,40,0.0,0.0,1,18.556701


In [478]:
#lp for finding team
data = D50L50.copy();
player = {str(i): data['name'][i] for i in range(data.shape[0])}
point = {str(i): data['ppm'][i] for i in range(data.shape[0])} 
cost = {str(i): data['credits'][i] for i in range(data.shape[0])}
gk = {str(i): 1 if data['pos_detail'][i] == 'goalkeeper' else 0 for i in range(data.shape[0])}
defender = {str(i): 1 if data['pos_detail'][i] == 'defender' else 0 for i in range(data.shape[0])}
midfielder = {str(i): 1 if data['pos_detail'][i] == 'midfielder' else 0 for i in range(data.shape[0])}
striker = {str(i): 1 if data['pos_detail'][i] == 'striker' else 0 for i in range(data.shape[0])}
club1 = {str(i): 1 if data['club'][i] == club1 else 0 for i in range(data.shape[0])}
club2 = {str(i): 1 if data['club'][i] == club2 else 0 for i in range(data.shape[0])}
xi = {str(i): 1 for i in range(data.shape[0])}

prob = LpProblem("Fantasy Football",LpMaximize)

player_index = LpVariable.dicts("Players",player,0,1,LpBinary)

# objective function
prob += lpSum([point[i]*player_index[i] for i in player]), "Total Cost"

# constraints
prob += lpSum([player_index[i] for i in player]) == 11, "Total 11 Players"
prob += lpSum([cost[i] * player_index[i] for i in player]) <= 100.0, "Total Cost"
prob += lpSum([gk[i] * player_index[i] for i in player]) == 1, "Only 1 GK"
prob += lpSum([defender[i] * player_index[i] for i in player]) >= 3, "Min 3 DEF"
prob += lpSum([defender[i] * player_index[i] for i in player]) <= 5, "Max 5 DEF"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) >= 3, "Min 3 MID"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) <= 5, "Max 5 MID"
prob += lpSum([striker[i] * player_index[i] for i in player]) >= 1, "Min 1 STR"
prob += lpSum([striker[i] * player_index[i] for i in player]) <= 3, "Max 3 STR"
prob += lpSum([club1[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club1"
prob += lpSum([club2[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club2"

# solve
status = prob.solve()
print(status)

1


/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [479]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [480]:
selection = {}
for v in prob.variables():
    index = int(v.name.split("_")[1])
    selection[index] = v.varValue

In [481]:
data['integer_programming'] = 0.0
for i in selection:
    data.loc[i, 'integer_programming'] = selection[i]

In [482]:
XI = data[data['integer_programming'] == 1.0]
TOP2 = XI['ppm'].nlargest(2)
CAPTAIN_POINTS = TOP2.iloc[0]
CAPTAIN = XI['name'][XI.ppm==CAPTAIN_POINTS]
VICECAPTAIN_POINTS = TOP2.iloc[1]
VICECAPTAIN = XI['name'][XI.ppm==VICECAPTAIN_POINTS]
PLAYER_POINTS = XI['ppm'].sum() 
TOTAL_POINTS = PLAYER_POINTS + CAPTAIN_POINTS + VICECAPTAIN_POINTS/2
TOTAL_COST = XI['credits'].sum()
TOTAL_PLAYERS = XI.shape[0]
print("Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

Total Points: 828.3750021183176, Raw Player Points: 669.8381264800965, Cost: 99.5, Captain: Haris Seferovic, Vice-Captain: Pablo Sarabia


In [483]:
print("50 DREAM : 50 LEAGUE \n")
print(XI[['name','pos_detail','club','credits','ppm',]]
 .sort_values(['pos_detail'])
 .reset_index(drop=True))

print("\n Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

50 DREAM : 50 LEAGUE 

                 name  pos_detail club  credits         ppm
0   César Azpilicueta    defender  SPN      9.0   45.919598
1          Jordi Alba    defender  SPN      8.5   51.807899
2         Nico Elvedi    defender  SUI      8.5   42.079918
3         Yann Sommer  goalkeeper  SUI      8.5    8.788269
4       Pablo Sarabia  midfielder  SPN      9.5  103.822485
5        Steven Zuber  midfielder  SUI      9.0   58.783055
6     Sergio Busquets  midfielder  SPN      8.5   42.665927
7        Remo Freuler  midfielder  SUI      8.5   43.008696
8     Haris Seferovic     striker  SUI     10.0  106.625633
9       Álvaro Morata     striker  SPN     10.0   84.312977
10      Ferran Torres     striker  SPN      9.5   82.023669

 Total Points: 828.3750021183176, Raw Player Points: 669.8381264800965, Cost: 99.5, Captain: Haris Seferovic, Vice-Captain: Pablo Sarabia


75 DREAM 25 LEAGUE

In [484]:
D75L25 = final_data.copy()
D75L25['ppm'] = ((0.75*final_data['dream11_points']) + (0.25*final_data['dream_points'])) / ((0.75*(final_data['mins_played_dream11'])/90) + (0.25*((final_data['mins_played_league'])/90)))
D75L25['ppm'] = D75L25['ppm'].fillna(0)
D75L25

,name,credits,pos_detail,club,mins_played_dream11,dream11_points,mins_played_league,dream_points,is_playing,ppm
0,Unai Simón,8.5,goalkeeper,SPN,390,99,3330.0,174.8,1,9.436000
1,Yann Sommer,8.5,goalkeeper,SUI,390,82,2713.0,221.0,1,10.824105
2,César Azpilicueta,9.0,defender,SPN,197,130,2191.0,1088.4,1,47.827462
3,Aymeric Laporte,8.5,defender,SPN,390,205,1343.0,404.8,1,36.522881
4,Jordi Alba,8.5,defender,SPN,312,164,3030.0,1759.8,1,51.099849
5,Pau Torres,8.5,defender,SPN,247,134,2970.0,877.6,1,31.033145
6,Manuel Akanji,8.5,defender,SPN,390,90,2350.0,1126.6,1,35.708523
7,Nico Elvedi,8.5,defender,SUI,390,73,2538.0,1296.0,1,36.771845
8,Ricardo Rodríguez,8.5,defender,SUI,357,46,1245.0,397.2,1,20.797927
9,Silvan Widmer,8.5,defender,SUI,194,40,0.0,0.0,1,18.556701


In [485]:
#lp for finding team
data = D75L25.copy();
player = {str(i): data['name'][i] for i in range(data.shape[0])}
point = {str(i): data['ppm'][i] for i in range(data.shape[0])} 
cost = {str(i): data['credits'][i] for i in range(data.shape[0])}
gk = {str(i): 1 if data['pos_detail'][i] == 'goalkeeper' else 0 for i in range(data.shape[0])}
defender = {str(i): 1 if data['pos_detail'][i] == 'defender' else 0 for i in range(data.shape[0])}
midfielder = {str(i): 1 if data['pos_detail'][i] == 'midfielder' else 0 for i in range(data.shape[0])}
striker = {str(i): 1 if data['pos_detail'][i] == 'striker' else 0 for i in range(data.shape[0])}
club1 = {str(i): 1 if data['club'][i] == club1 else 0 for i in range(data.shape[0])}
club2 = {str(i): 1 if data['club'][i] == club2 else 0 for i in range(data.shape[0])}
xi = {str(i): 1 for i in range(data.shape[0])}

prob = LpProblem("Fantasy Football",LpMaximize)

player_index = LpVariable.dicts("Players",player,0,1,LpBinary)

# objective function
prob += lpSum([point[i]*player_index[i] for i in player]), "Total Cost"

# constraints
prob += lpSum([player_index[i] for i in player]) == 11, "Total 11 Players"
prob += lpSum([cost[i] * player_index[i] for i in player]) <= 100.0, "Total Cost"
prob += lpSum([gk[i] * player_index[i] for i in player]) == 1, "Only 1 GK"
prob += lpSum([defender[i] * player_index[i] for i in player]) >= 3, "Min 3 DEF"
prob += lpSum([defender[i] * player_index[i] for i in player]) <= 5, "Max 5 DEF"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) >= 3, "Min 3 MID"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) <= 5, "Max 5 MID"
prob += lpSum([striker[i] * player_index[i] for i in player]) >= 1, "Min 1 STR"
prob += lpSum([striker[i] * player_index[i] for i in player]) <= 3, "Max 3 STR"
prob += lpSum([club1[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club1"
prob += lpSum([club2[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club2"

# solve
status = prob.solve()
print(status)

1


/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [486]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [487]:
selection = {}
for v in prob.variables():
    index = int(v.name.split("_")[1])
    selection[index] = v.varValue

In [488]:
data['integer_programming'] = 0.0
for i in selection:
    data.loc[i, 'integer_programming'] = selection[i]

In [489]:
XI = data[data['integer_programming'] == 1.0]
TOP2 = XI['ppm'].nlargest(2)
CAPTAIN_POINTS = TOP2.iloc[0]
CAPTAIN = XI['name'][XI.ppm==CAPTAIN_POINTS]
VICECAPTAIN_POINTS = TOP2.iloc[1]
VICECAPTAIN = XI['name'][XI.ppm==VICECAPTAIN_POINTS]
PLAYER_POINTS = XI['ppm'].sum() 
TOTAL_POINTS = PLAYER_POINTS + CAPTAIN_POINTS + VICECAPTAIN_POINTS/2
TOTAL_COST = XI['credits'].sum()
TOTAL_PLAYERS = XI.shape[0]
print("Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

Total Points: 797.7777904482776, Raw Player Points: 646.0787640156173, Cost: 99.5, Captain: Pablo Sarabia, Vice-Captain: Haris Seferovic


In [490]:
print("75 DREAM : 25 LEAGUE \n")
print(XI[['name','pos_detail','club','credits','ppm',]]
 .sort_values(['pos_detail'])
 .reset_index(drop=True))

print("\n Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

75 DREAM : 25 LEAGUE 

                 name  pos_detail club  credits         ppm
0   César Azpilicueta    defender  SPN      9.0   47.827462
1          Jordi Alba    defender  SPN      8.5   51.099849
2         Nico Elvedi    defender  SUI      8.5   36.771845
3         Yann Sommer  goalkeeper  SUI      8.5   10.824105
4     Xherdan Shaqiri  midfielder  SUI      9.5   51.551192
5       Pablo Sarabia  midfielder  SPN      9.5  102.805369
6        Steven Zuber  midfielder  SUI      9.0   61.135550
7     Sergio Busquets  midfielder  SPN      8.5   41.720564
8     Haris Seferovic     striker  SUI     10.0   97.787315
9       Ferran Torres     striker  SPN      9.5   80.183496
10       Breel Embolo     striker  SUI      9.0   64.372018

 Total Points: 797.7777904482776, Raw Player Points: 646.0787640156173, Cost: 99.5, Captain: Pablo Sarabia, Vice-Captain: Haris Seferovic


DREAM 25 LEAGUE 75

In [491]:
D25L75 = final_data.copy()
D25L75['ppm'] = ((0.25*final_data['dream11_points']) + (0.75*final_data['dream_points'])) / ((0.25*(final_data['mins_played_dream11'])/90) + (0.75*((final_data['mins_played_league'])/90)))
D25L75['ppm'] = D25L75['ppm'].fillna(0)
D25L75

,name,credits,pos_detail,club,mins_played_dream11,dream11_points,mins_played_league,dream_points,is_playing,ppm
0,Unai Simón,8.5,goalkeeper,SPN,390,99,3330.0,174.8,1,5.405202
1,Yann Sommer,8.5,goalkeeper,SUI,390,82,2713.0,221.0,1,7.861414
2,César Azpilicueta,9.0,defender,SPN,197,130,2191.0,1088.4,1,45.135598
3,Aymeric Laporte,8.5,defender,SPN,390,205,1343.0,404.8,1,28.908350
4,Jordi Alba,8.5,defender,SPN,312,164,3030.0,1759.8,1,52.106573
5,Pau Torres,8.5,defender,SPN,247,134,2970.0,877.6,1,27.193622
6,Manuel Akanji,8.5,defender,SPN,390,90,2350.0,1126.6,1,41.973387
7,Nico Elvedi,8.5,defender,SUI,390,73,2538.0,1296.0,1,44.538981
8,Ricardo Rodríguez,8.5,defender,SUI,357,46,1245.0,397.2,1,27.219941
9,Silvan Widmer,8.5,defender,SUI,194,40,0.0,0.0,1,18.556701


In [492]:
#lp for finding team
data = D25L75.copy();
player = {str(i): data['name'][i] for i in range(data.shape[0])}
point = {str(i): data['ppm'][i] for i in range(data.shape[0])} 
cost = {str(i): data['credits'][i] for i in range(data.shape[0])}
gk = {str(i): 1 if data['pos_detail'][i] == 'goalkeeper' else 0 for i in range(data.shape[0])}
defender = {str(i): 1 if data['pos_detail'][i] == 'defender' else 0 for i in range(data.shape[0])}
midfielder = {str(i): 1 if data['pos_detail'][i] == 'midfielder' else 0 for i in range(data.shape[0])}
striker = {str(i): 1 if data['pos_detail'][i] == 'striker' else 0 for i in range(data.shape[0])}
club1 = {str(i): 1 if data['club'][i] == club1 else 0 for i in range(data.shape[0])}
club2 = {str(i): 1 if data['club'][i] == club2 else 0 for i in range(data.shape[0])}
xi = {str(i): 1 for i in range(data.shape[0])}

prob = LpProblem("Fantasy Football",LpMaximize)

player_index = LpVariable.dicts("Players",player,0,1,LpBinary)

# objective function
prob += lpSum([point[i]*player_index[i] for i in player]), "Total Cost"

# constraints
prob += lpSum([player_index[i] for i in player]) == 11, "Total 11 Players"
prob += lpSum([cost[i] * player_index[i] for i in player]) <= 100.0, "Total Cost"
prob += lpSum([gk[i] * player_index[i] for i in player]) == 1, "Only 1 GK"
prob += lpSum([defender[i] * player_index[i] for i in player]) >= 3, "Min 3 DEF"
prob += lpSum([defender[i] * player_index[i] for i in player]) <= 5, "Max 5 DEF"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) >= 3, "Min 3 MID"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) <= 5, "Max 5 MID"
prob += lpSum([striker[i] * player_index[i] for i in player]) >= 1, "Min 1 STR"
prob += lpSum([striker[i] * player_index[i] for i in player]) <= 3, "Max 3 STR"
prob += lpSum([club1[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club1"
prob += lpSum([club2[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club2"

# solve
status = prob.solve()
print(status)

1


/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [493]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [494]:
selection = {}
for v in prob.variables():
    index = int(v.name.split("_")[1])
    selection[index] = v.varValue

In [495]:
data['integer_programming'] = 0.0
for i in selection:
    data.loc[i, 'integer_programming'] = selection[i]

In [496]:
XI = data[data['integer_programming'] == 1.0]
TOP2 = XI['ppm'].nlargest(2)
CAPTAIN_POINTS = TOP2.iloc[0]
CAPTAIN = XI['name'][XI.ppm==CAPTAIN_POINTS]
VICECAPTAIN_POINTS = TOP2.iloc[1]
VICECAPTAIN = XI['name'][XI.ppm==VICECAPTAIN_POINTS]
PLAYER_POINTS = XI['ppm'].sum() 
TOTAL_POINTS = PLAYER_POINTS + CAPTAIN_POINTS + VICECAPTAIN_POINTS/2
TOTAL_COST = XI['credits'].sum()
TOTAL_PLAYERS = XI.shape[0]
print("Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

Total Points: 846.1313542342291, Raw Player Points: 683.4115056346258, Cost: 99.0, Captain: Haris Seferovic, Vice-Captain: Pablo Sarabia


In [497]:
print("25 DREAM : 75 LEAGUE \n")
print(XI[['name','pos_detail','club','credits','ppm',]]
 .sort_values(['pos_detail'])
 .reset_index(drop=True))

print("\n Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

25 DREAM : 75 LEAGUE 

                 name  pos_detail club  credits         ppm
0   César Azpilicueta    defender  SPN      9.0   45.135598
1          Jordi Alba    defender  SPN      8.5   52.106573
2         Nico Elvedi    defender  SUI      8.5   44.538981
3         Yann Sommer  goalkeeper  SUI      8.5    7.861414
4       Pablo Sarabia  midfielder  SPN      9.5  104.297565
5        Steven Zuber  midfielder  SUI      9.0   57.362841
6     Sergio Busquets  midfielder  SPN      8.5   43.037228
7        Remo Freuler  midfielder  SUI      8.5   44.788347
8     Haris Seferovic     striker  SUI     10.0  110.571066
9       Álvaro Morata     striker  SPN     10.0   88.832083
10       Breel Embolo     striker  SUI      9.0   84.879811

 Total Points: 846.1313542342291, Raw Player Points: 683.4115056346258, Cost: 99.0, Captain: Haris Seferovic, Vice-Captain: Pablo Sarabia
